In [1]:
import heapq
import random, math

In [2]:
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.7
POPULATION_SIZE = 100
FITNESS = 0
TRUCKS = 5
DEPOT = None
CAPACITY = 100
INF = float("inf")

In [3]:
class PrioritySet(object):
    
    def __init__(self):
        self.heap = []
        self.set = set()

    def push(self, d):
        if not d in self.set:
            heapq.heappush(self.heap, d)
            self.set.add(d)

    def pop(self):
        d = heapq.heappop(self.heap)
        self.set.remove(d)
        return d

    def poop(self):
        d = self.heap[-1]
        self.heap = self.heap[:-1]
        self.set.remove(d)
        return d

    def size(self):
        return len(self.heap)

    def __str__(self):
        op = ""
        for i in self.heap:
            op += str(i[0]) + " : " + i[1].__str__()
            op += "\n"
        return op

    def __getitem__(self, index):
        return self.heap[index]


In [4]:
class Position:

    def __init__(self,x,y):
        self.x = x
        self.y = y		

    def __str__(self):
        return "(" + str(self.x) + ", " + str(self.y) + ") "

    def x_coor(self):
        return self.x

    def y_coor(self):
        return self.y

In [5]:
class Vehicle:

    def __init__(self, capacity):
        self.capacity = capacity

    def capacity(self):
        return self.capacity

In [6]:
class Node:

    pos = Position(-1,-1)
    demand = 0

    def __init__(self,name):
        self.name = name

    def setPosition(self,x,y):
        self.pos = Position(x, y)

    def setDemand(self,d):
        self.demand = d

    def __str__(self):
        return "(" + str(self.pos.x) + ", " + \
                   str(self.pos.y) + " )"

In [7]:
def copy(li):
    return [i for i in li]

def getProb():
    return random.random()

def get_random(li):
    index = random.randint(0, len(li)-1)
    return li[index]

def get_distance(cus1, cus2):
    # Euclideian
    #change to vincenty later
    dist = 0 
    dist = math.sqrt(((cus1.pos.x - cus2.pos.x) ** 2) + ((cus1.pos.y - cus2.pos.y) ** 2))
    return dist

def print_tuple(t):
    print( "0"), 
    for i in t:
        print(i),        
    print( "0"),
    print (" -> f: " + str(get_fitness(t)))

def print_population(p):
    for i in p:
        for c in i:
            print(c),
        print("\n")
    
def print_population_heap(p):
    count = 1
    for i in p:
        print (count, " )  ")
        print_tuple(i[1])
        count += 1
        print("\n")



In [8]:
def mutate(chromosome):

    temp = [i for i in chromosome]
    
    if getProb() < MUTATION_RATE:
        left = random.randint(1, len(temp) - 2)
        right = random.randint(left, len(temp) - 1)   
        temp[left], temp[right] = temp[right], temp[left]
    return temp


In [9]:

def crossover(a,b):

    if getProb() < CROSSOVER_RATE:
        left = random.randint(1, len(a) - 2)
        right = random.randint(left, len(a) - 1)
        # print left, " ", right
        c1 = [c for c in a[0:] if c not in b[left:right+1]]
        # print len(c1)
        a1 = c1[:left] + b[left:right+1] + c1[left:]
        # print len(p1)
        c2 = [c for c in b[0:] if c not in a[left:right+1]]
        b1 = c2[:left] + a[left:right+1] + c2[left:]
        return a1, b1
    
    return a, b


In [10]:

def get_fitness(li):
    
    num_custo = len(li)
    fitness = 0

    for i in range(num_custo - 1):
        fitness += get_distance(li[i], li[i+1])

    fitness += get_distance(DEPOT, li[0])
    fitness += get_distance(li[-1], DEPOT)

    # chk for valid capacity
    temp = copy(li)
    temp.insert(0,DEPOT)
    temp.append(DEPOT)
    valid = 1
    curr_demand = 0
    for i in range(len(temp)):
        if temp[i] == DEPOT and curr_demand > CAPACITY:
            fitness = INF
        elif temp[i] == DEPOT:
            curr_demand = 0
        else:
            curr_demand += temp[i].demand

    return fitness


In [11]:

def getPopulationFitness(p):
    
    h = PrioritySet()
    for i in p:
        h.push((get_fitness(i),i))
    return h

def create_new():

    TempSet = copy(Nodes)
    chromosome = []
    while len(TempSet) > 0:
        index = (int)(getProb() * len(TempSet))
        chromosome.append(TempSet.pop(index))

    return chromosome 


In [12]:

def Genetic_Algo():
    
    print ("POPULATION GENERATED... EVOLUTION BEGINING ...")
    minimum_chrom = h[0]
    print( "Curr Min: ", minimum_chrom[0])
    count = 0
    # while h[0][0] > 1800:
    while count < 1000:
        ax = h.pop()
        bx = h.pop()
        a,b = crossover(list(ax[1]),list(bx[1]))
        a = mutate(a)
        while get_fitness(a) == INF:
            a = create_new()
        b = mutate(b)
        while get_fitness(b) == INF:
            b = create_new()
        if get_fitness(a) != INF:
            h.push((get_fitness(a),tuple(a)))
        else:
            h.push(ax)
        if get_fitness(b) != INF:
            h.push((get_fitness(b),tuple(b)))
        else:
            h.push(bx)

        while h.size() < POPULATION_SIZE:
            TempSet = copy(Nodes)
            chromosome = []
            count += 1
            while len(TempSet) > 0:
                index = (int)(getProb() * len(TempSet))
                chromosome.append(TempSet.pop(index))
            h.push((get_fitness(chromosome),tuple(chromosome)))
        count = count + 1
        
        if count % 1000 == 0:
            print (count),
            print (" Generation done")

        if h[0][0] < minimum_chrom[0]:
            minimum_chrom = h[0] 
            print ("CurrMin: ",)
            print (minimum_chrom[0])
    
    print_tuple(minimum_chrom[1])
    print (count)
    


In [13]:

def initialize_population():

    while len(population) < POPULATION_SIZE:
        TempSet = copy(Nodes)
        chromosome = []
        while len(TempSet) > 0:
            index = (int)(getProb() * len(TempSet))
            chromosome.append(TempSet.pop(index))

        if get_fitness(chromosome) != INF:
            population.add(tuple(chromosome))
    

In [14]:

def create_data_array():

    locations = [(43.137407,77.598233),(43.136945,-77.598493),
                (43.134420,-77.602399),(43.137430,-77.603818),
                (43.136636,-77.601178),(43.135769,-77.598343),
                (43.138067,-77.600400),(43.137609,-77.598897),
                (43.138074,-77.603483),(43.137271,-77.600835),
                (43.136415,-77.597977),(43.136007,-77.601529),
                (43.135133,-77.598700),(43.135553,-77.598466),
                (43.134500,-77.599068),(43.135347,-77.601880),
                (43.133946,-77.603805),(43.136355,-77.598010)]


    demands =  [0, 19, 21, 6, 19, 
                7, 12, 16, 6, 16, 
                8, 14, 21, 16, 3, 
                22, 18, 19, 1, 24, 
                8, 12, 4, 8, 24, 
                24, 2, 20, 15, 2]


    for i in range(1,len(locations)):
        c = Node(i)
        c.setPosition(locations[i][0],locations[i][1])
        c.setDemand(demands[i])
        Nodes.append(c)
    
    i = 0
    c = Node(i)
    c.setPosition(locations[i][0],locations[i][1])
    c.setDemand(demands[i])
    global DEPOT
    DEPOT = c

    for j in range(TRUCKS-1):
        Nodes.append(DEPOT)


In [15]:

Nodes = []
population = set()

if __name__ == '__main__':
    create_data_array()
    initialize_population()
    # print_population(population)
    h = getPopulationFitness(population)
    # print_population_heap(h)
    Genetic_Algo()

POPULATION GENERATED... EVOLUTION BEGINING ...
Curr Min:  931.2244472619477
CurrMin: 
931.2217069745512
CurrMin: 
931.220819884666
1000
 Generation done
0
(43.135553, -77.598466 )
(43.135133, -77.5987 )
(43.1345, -77.599068 )
(43.135347, -77.60188 )
(43.138067, -77.6004 )
(43.13743, -77.603818 )
(43.136415, -77.597977 )
(43.137407, 77.598233 )
(43.135769, -77.598343 )
(43.137609, -77.598897 )
(43.136007, -77.601529 )
(43.136355, -77.59801 )
(43.137407, 77.598233 )
(43.137407, 77.598233 )
(43.136945, -77.598493 )
(43.13442, -77.602399 )
(43.133946, -77.603805 )
(43.138074, -77.603483 )
(43.137271, -77.600835 )
(43.136636, -77.601178 )
0
 -> f: 931.220819884666
1000
